In [1]:
import torch
import torch.nn as nn

from lib.Exercise1_1 import LQRSolver
from torch.utils.data import TensorDataset, DataLoader
class DGMNN(nn.Module):
    def __init__(self):
        super(DGMNN, self).__init__()
        self.layer1 = nn.Linear(3, 50)  
        self.layer2 = nn.Linear(50, 50)
        self.tanh = nn.Tanh()
        self.output = nn.Linear(50, 1)
        
    def forward(self, x):
        x = self.tanh(self.layer1(x))
        x = self.tanh(self.layer2(x))
        return self.output(x)



In [2]:
def get_hessian(grad,x):
    Hessian = torch.tensor([])
    
    for i in range(len(x)):
        hessian = torch.tensor([])
        for j in range(len(grad[i])):
            u_xxi = torch.autograd.grad(grad[i][j], x, grad_outputs=torch.ones_like(grad[i][j]), retain_graph=True,create_graph=True, allow_unused=True)[0]           
            hessian = torch.cat((hessian, u_xxi[i].unsqueeze(0)))
        Hessian = torch.cat((Hessian, hessian.unsqueeze(0)),dim = 0)
    return Hessian

def pde_residual(model, t, x):
    
    input = torch.cat((t.unsqueeze(1), x),dim=1)
    
    u = model(input)

    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True, retain_graph=True)[0]
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True, retain_graph=True)[0]

    u_xx = get_hessian(u_x,x)

    residual = u_t + 0.5 * torch.einsum('bii->b', sigma @ sigma.transpose(1,2) @ u_xx) + (u_x.unsqueeze(1) @ (H @ x.unsqueeze(1).transpose(1,2)) + u_x.unsqueeze(1) @ M @ alpha + x.unsqueeze(1) @ C @ x.unsqueeze(1).transpose(1,2) + alpha.transpose(1,2) @ D @ alpha).squeeze()
    return residual

def boundary_condition(model,t, x):
    # 定义边界条件的残差
    
    T_input = T * torch.ones_like(t)

    input = torch.cat((T_input.unsqueeze(1), x),dim=1)
    u = model(input)

    return u - (x.unsqueeze(1) @ R @ x.unsqueeze(1).transpose(1,2)).squeeze()

def total_residual(model, t, x):
    # 计算内部PDE残差
    residual_loss = pde_residual(model, t, x).pow(2).mean()
    # 计算边界条件的残差
    boundary_loss = boundary_condition(model,t,x).pow(2).mean()
    
    return residual_loss + boundary_loss


In [3]:
def new_data(num_samples):
    #num_samples = 10000
    t_samples = T * torch.rand(num_samples, dtype=torch.double, requires_grad=True)
    x_ends = torch.tensor([-3,3], dtype = torch.double)
    x_samples = x_ends[0] + (x_ends[1]- x_ends[0]) * torch.rand(num_samples , 2, dtype=torch.double, requires_grad=True)
    return t_samples,x_samples

In [4]:
# Define matrices for LQR problem
H = torch.tensor([[1.2, 0.8], [-0.6, 0.9]], dtype=torch.double)
M = torch.tensor([[0.5,0.7], [0.3,1.0]], dtype=torch.double)
sigma = torch.tensor([[[0.8],[1.1]]], dtype=torch.double)
alpha = torch.tensor([[[1],[1]]], dtype=torch.double)
C = torch.tensor([[1.6, 0.0], [0.0, 1.1]], dtype=torch.double)
D = torch.tensor([[0.5, 0.0], [0.0, 0.7]], dtype=torch.double)
R = torch.tensor([[0.9, 0.0], [0.0, 1.0]], dtype=torch.double)
T = torch.tensor(1.0, dtype=torch.double)

solver = LQRSolver(H, M, sigma, C, D, R, T=T, method="euler")


In [5]:
model_DGM = DGMNN().double()

# Prepare for training
optimizer_DGM = torch.optim.Adam(model_DGM.parameters(), lr=0.001)
epoch_losses = []

batch_size = 30
epochs = 500

for batch in range(batch_size):
    print(f'Batch {batch+1}/{batch_size}'+'\n')
    
    t_data,x_data = new_data(50)
    dataset = TensorDataset(t_data,x_data)
    dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

    for epoch in range(epochs):

        model_DGM.train()
        total_loss = 0
        
        for batch_idx, (t_data_,x_data_) in enumerate(dataloader):
            optimizer_DGM.zero_grad()
            loss = total_residual(model_DGM, t_data_, x_data_) 
            loss.backward(retain_graph=True)
            optimizer_DGM.step()
            total_loss += loss.item()
        epoch_losses.append(total_loss / len(dataloader))

        if epoch == 0:
            print(f'Epoch {epoch+1}/{epochs} \t Loss: {total_loss / len(dataloader)}')
        if (epoch+1) % 100 == 0:
            print(f'Epoch {epoch+1}/{epochs} \t Loss: {total_loss / len(dataloader)}')

print('\n')
model_DGM.eval()

Batch 1/3

Epoch 1/1000 	 Loss: 134.26704548228946
Epoch 100/1000 	 Loss: 48.75391410236446
Epoch 200/1000 	 Loss: 21.128226349154012


KeyboardInterrupt: 

In [208]:
# Initialization

t_ends = [0.1,0.9]
t_num = 3
x_ends = [[0.1,1.8],[0.1,2.7]]
x_num = [100,100] # alleviate the calculation workload.

# load_interval_setting = torch.load('Exercise1_2/value_numerical/?x?/'+'interval_setting.pt')
# t_ends = load_interval_setting['t_ends']
# t_num = load_interval_setting['t_num']
# x_ends = load_interval_setting['x_ends']
# x_num = load_interval_setting['x_num']


# Establish meshgrid structure from setting.

t_batch_i = torch.linspace(t_ends[0],t_ends[1],t_num,dtype=torch.double)
t_batch = t_batch_i.repeat_interleave(x_num[0]*x_num[1])

x1 = torch.linspace(x_ends[0][0],x_ends[0][1],x_num[0],dtype=torch.double)
x2 = torch.linspace(x_ends[1][0],x_ends[1][1],x_num[1],dtype=torch.double)

x_batch_i = torch.cartesian_prod(x1, x2).unsqueeze(1)

X1 = x_batch_i[:, 0, 0].view(x_num[0], x_num[1])
X2 = x_batch_i[:, 0, 1].view(x_num[0], x_num[1])

x_batch = torch.cartesian_prod(x1, x2).unsqueeze(1).repeat(t_num, 1, 1)

value_numerical = solver.value_function(t_data_,x_data.unsqueeze(1))


In [222]:
x_data

tensor([[ 0.6843, -0.7246],
        [ 0.6360, -0.5343],
        [-1.6150, -1.7099],
        [-0.1671, -1.9113],
        [-0.6182,  0.9861]], dtype=torch.float64, grad_fn=<AddBackward0>)

In [225]:
value_numerical = solver.value_function(t_data.detach(),x_data.detach().unsqueeze(1))

In [230]:
A = model_DGM(torch.cat((t_data.unsqueeze(1), x_data.squeeze(1)),dim=1))

In [231]:
A

tensor([[3.1192],
        [3.0309],
        [3.4098],
        [3.7110],
        [4.6666]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

In [232]:
value_numerical

tensor([4.3155, 3.0054, 7.5393, 5.5838, 8.6400], dtype=torch.float64)

In [229]:

B = torch.abs(A.squeeze() - value_numerical)
B

RuntimeError: The size of tensor a (30000) must match the size of tensor b (5) at non-singleton dimension 0

In [37]:
t_samples.shape

x_samples.shape

torch.Size([1000, 2])

In [12]:
value_numerical.shape

torch.Size([30000])

In [210]:
# Assume we're evaluating at specific t and across a grid in x
t_eval = torch.tensor([[0.5]])  # Example: Evaluate at t=0.5
x_eval_grid = ...  # Create a grid of x values within the domain

with torch.no_grad():
    u_pred = model(t_eval, x_eval_grid)
    # Visualization or comparison code here


TypeError: expected Tensor as element 1 in argument 0, but got ellipsis